In [5]:
import geopandas as gpd
import rasterio as rio
from pathlib import Path

In [4]:
with rio.open('../data/processed/release_areas/release_areas.sdat') as f:
    band = f.read()
    print(f"CRS: {f.crs}")
    print(f"Bounds: {f.bounds}")

CRS: PROJCS["MGI / Austria GK Central",GEOGCS["MGI",DATUM["Militar-Geographische_Institut",SPHEROID["Bessel 1841",6377397.155,299.1528128],AUTHORITY["EPSG","6312"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",13.3333333333333],PARAMETER["scale_factor",1],PARAMETER["false_easting",0],PARAMETER["false_northing",-5000000],UNIT["metre",1],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","31255"]]
Bounds: BoundingBox(left=-50000.25, bottom=207999.75, right=-47499.75, top=216000.25)


In [9]:
files = {
    'release': Path('../data/processed/release_areas/release_areas.sdat'),
    'simulation': Path('../data/processed/simulations/sim_003/process_area.sdat')
}

for name, path in files.items():
    if path.exists():
        with rio.open(path) as src:
            print(f"{name}:")
            print(f"  CRS: {src.crs}")
            print(f"  Bounds: {src.bounds}")
            print(f"  Shape: {src.shape}")
    else:
        print(f"{name}: FILE NOT FOUND!")

release:
  CRS: PROJCS["MGI / Austria GK Central",GEOGCS["MGI",DATUM["Militar-Geographische_Institut",SPHEROID["Bessel 1841",6377397.155,299.1528128],AUTHORITY["EPSG","6312"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",13.3333333333333],PARAMETER["scale_factor",1],PARAMETER["false_easting",0],PARAMETER["false_northing",-5000000],UNIT["metre",1],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","31255"]]
  Bounds: BoundingBox(left=-50000.25, bottom=207999.75, right=-47499.75, top=216000.25)
  Shape: (16001, 5001)
simulation:
  CRS: PROJCS["MGI / Austria GK Central",GEOGCS["MGI",DATUM["Militar-Geographische_Institut",SPHEROID["Bessel 1841",6377397.155,299.1528128],AUTHORITY["EPSG","6312"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",13.3333333333333],PARAMETER

In [10]:
import sys
from pathlib import Path
# sys.path.insert(0, str(Path(__file__).parent.parent))
sys.path.append('../')
from config.database import load_config
import psycopg2
from datetime import datetime, timedelta
from config.settings import settings 

In [11]:
database = '../database/database.ini'
config = load_config(filename=database)
with psycopg2.connect(**config) as conn:
    with conn.cursor() as cur:
        cur.execute("""
            SELECT timestamp, precipitation_mm, temperature_c, humidity_percent
            FROM weather_data_hourly
            WHERE timestamp >= %s
            ORDER BY timestamp DESC
        """, (datetime.now() - timedelta(hours=24),))
        
        rows = cur.fetchall()
        results = [{
            'timestamp': r[0],
            'precipitation_mm': r[1],
            'temperature_c': r[2],
            'humidity_percent': r[3]
        } for r in rows]
    